# **Creating comparison results with Godunov for 2 dimensional Riemann problems**
This notebook is for easily creating comparison results with respect to Godunovs scheme to use in the thesis. The code is based on code from the development stage, found in `create_results_2dim_newInit.ipynb` which is a notebook used for creation of main results. However, in this notebook we are only interested in the relative errors, for comparison. On the other hand, just to be safe, we will save results in case they are needed.

## **How to produce results**
1. Choose parameters. Be wise on choice of destination to avoid overwriting.
2. Restart kernel and run all cells.

**NB**: `Make sure not to overwrite wanted material, thus choose destination and filename with care`**!!!**

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from datetime import datetime
print("Last run:",datetime.today())

Last run: 2021-03-05 13:48:03.016733


## Imports

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

#from riemannsolver import data2d
#from riemannsolver import net_mlp2d as network
#from riemannsolver import god_mlp2d_newParam as god_aprox
from riemannsolver import netplot
from riemannsolver import function

from riemannsolver import godunov_2d as god_2d

from IPython.display import HTML
from tqdm import tqdm_notebook as tqdm

## **Parameters**

#### Flux and derivative in x-direction and y-direction

In [4]:
f = lambda x: x**2/2
dfdu = lambda x: x

g = lambda x: x**2/2
dgdu = lambda x: x

#### Destination and filename 

In [5]:
destination = "res/2dim_newInit/burgers/godunov_mesh100"
name = "godunov_mesh100"

#### Mesh- and method-parameters
To enhance performance, set small $N_y$-value if y-direction in initial data is constant.
* $N_x$ - mesh size in x-direction
* $N_y$ - mesh size in y-direction
* T - temporal maximum
* C - Courant coefficient

In [6]:
Nx = 100
Ny = 100
T = 0.5
C = 0.5

xmin, xmax = -1, 1
ymin, ymax = -1, 1

## **Approximate by Godunov**

In [7]:
bounds = [
    'periodic',
    'neumann'
]
labels = [
    'splash',
    'cone_box'
]
init_funcs = [
    function.InitialFunc('watersplash').func,
    function.InitialFunc('circ_square').func
]
initial_t0 = []
result_half = []
result = []
for init_func, bound, label in zip(init_funcs, bounds, labels):
    print("solving "+label)
    god = god_2d.Godunov(
        f=f, dfdU=dfdu, g=g, dgdU=dgdu, U0=init_func, 
        x_min=xmin, x_max=xmax, Nx=Nx, 
        y_min=ymin, y_max=ymax, Ny=Ny,
        bnd_cond=bound, T=T, C=C
    )
    god.solve
    initial_t0.append(god.u[0])
    result_half.append(god.u[len(god.u)//2])
    result.append(god.u[-1])
    
    print("length of u: ",len(god.u))
    
    mesh4vid = god.mesh
    u4vid  = torch.zeros((len(god.u),god.u[0].size(0),god.u[0].size(1)))
    for i in range(len(god.u)):
        u4vid[i] = god.u[i]

solving splash



length of u:  50
solving cone_box



length of u:  71


## **Plot**

In [8]:
y = torch.linspace(-1, 1, result[0].size(0), dtype=torch.float64)
x = torch.linspace(-1, 1, result[0].size(1), dtype=torch.float64)
y_mesh,x_mesh = torch.meshgrid(y,x)
mesh = torch.stack((y_mesh,x_mesh),axis=2)
x, y = mesh[:,:,0].numpy(), mesh[:,:,1].numpy()

init_t0 = [res.numpy() for res in initial_t0]

u_ref = [
    torch.load('res/2dim_newInit/burgers/exact/u_reference_solution_2D_waterSplashInitial_1000x1000.pt'),
    torch.load('res/2dim_newInit/burgers/exact/u_reference_solution_2D_coneBoxInitial_1000x1000.pt')
]

res_scaled = [torch.zeros(u_ref[0][-1].size()),torch.zeros(u_ref[0][-1].size())]
res_half_scaled = [torch.zeros(u_ref[0][-1].size()),torch.zeros(u_ref[0][-1].size())]

scale_param_x=1000//Nx
scale_param_y=1000//Ny
for i in range(result[0].size(0)):
    for j in range(result[0].size(1)):
        res_scaled[0][i*scale_param_x:(i+1)*scale_param_x,j*scale_param_y:(j+1)*scale_param_y] = result[0][i,j]
        res_half_scaled[0][i*scale_param_x:(i+1)*scale_param_x,j*scale_param_y:(j+1)*scale_param_y] = result_half[0][i,j]
        res_scaled[1][i*scale_param_x:(i+1)*scale_param_x,j*scale_param_y:(j+1)*scale_param_y] = result[1][i,j]
        res_half_scaled[1][i*scale_param_x:(i+1)*scale_param_x,j*scale_param_y:(j+1)*scale_param_y] = result_half[1][i,j]

z_half = [res.numpy() for res in res_half_scaled]
z = [res.numpy() for res in res_scaled]
        
for i,zi,zi_half,label in zip(range(len(z)),z,z_half, labels):
    # Half temporal:
    fig = plt.figure(figsize=(5,5))
    im = plt.imshow(
        zi_half.T,
        cmap='coolwarm',
        extent=[-1,1,-1,1],
        vmin=np.min(init_t0[i])-0.01,
        vmax=np.max(init_t0[i])+0.01
    )
    tick = np.linspace(-1, 1, 5)
    ratio = (zi_half.shape[0]/zi_half.shape[1])
    plt.colorbar(im, pad=0.04, fraction=0.046*ratio)
    plt.xticks(ticks=tick,labels=tick)
    plt.yticks(ticks=tick,labels=tick)
    plt.xlabel('x')
    plt.ylabel('y', rotation=0)
    plt.savefig(destination+'/'+label+'_network_color_'+name+'_T025.pdf',format='pdf')
    plt.close()
    
    re = float(
        np.sqrt(
            np.sum( (zi_half-np.array(u_ref[i][len(u_ref[i])//2]))**2 )/np.sum( (np.array(u_ref[i][len(u_ref[i])//2]))**2 )
        )
    )
    print("Relative error t=T/2 - godunov vs network - ",labels[i],": ",re)
    
    # Full temporal:
    fig = plt.figure(figsize=(5,5))
    im = plt.imshow(
        zi.T,
        cmap='coolwarm',
        extent=[-1,1,-1,1],
        vmin=np.min(init_t0[i])-0.01,
        vmax=np.max(init_t0[i])+0.01
    )
    tick = np.linspace(-1, 1, 5)
    ratio = (zi.shape[0]/zi.shape[1])
    plt.colorbar(im, pad=0.04, fraction=0.046*ratio)
    plt.xticks(ticks=tick,labels=tick)
    plt.yticks(ticks=tick,labels=tick)
    plt.xlabel('x')
    plt.ylabel('y', rotation=0)
    plt.savefig(destination+'/'+label+'_network_color_'+name+'_T05.pdf',format='pdf')
    plt.close()
    
    re = float(
        np.sqrt(
            np.sum( (zi-np.array(u_ref[i][-1]))**2 )/np.sum( (np.array(u_ref[i][-1]))**2 )
        )
    )
    print("Relative error t=T - godunov vs network - ",labels[i],": ",re)
    
    # Animation:
    flt = netplot.Surface(mesh=mesh4vid, z=u4vid)
    flt.save_anim_surface('circ_square_network_anim_surface', folder=destination+'/')
    flt.save_anim_color('circ_square_network_anim_color', folder=destination+'/')
HTML(flt.get_anim_color.to_html5_video())

Relative error t=T/2 - godunov vs network -  splash :  0.27516137848338434
Relative error t=T - godunov vs network -  splash :  0.3376716105613981
Relative error t=T/2 - godunov vs network -  cone_box :  0.17199645926742946
Relative error t=T - godunov vs network -  cone_box :  0.2232786225115888
